# SARSA & Q - Learning

# 연습데이터 : Taxi
![좋은거](https://storage.googleapis.com/lds-media/images/Reinforcement_Learning_Taxi_Env.width-1200.png)

**학습목표**

1. SARSA의 업데이트 맛 본다.
2. Q-learning의 업데이트를 맛 본다.
3. 일단 코드 짤 수 있다.

# 필요 라이브러리 불러오기

1. 이런 연습에서는 딱히 비디오 영상이 필요하지 않다.

In [1]:
!pip install gym

In [2]:
import numpy as np
import gym

# 사용할 환경 불러오기
[택시!](https://gym.openai.com/envs/Taxi-v3/)


In [4]:
env = gym.make('Taxi-v3')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation space: Discrete(500)
action space: Discrete(6)


# SARSA와, Q-learning 준비 해보기

**핵심 코드**
1. SARSA : S,A,R,S,A 가 끝나면 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha * ((r + gama* Q[s1,a1]) - Q[s,a])
```
2. Q-Learning : S,A,R,S 후 max A, 그다음 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha*(r + gama*np.max(Q[s1,:]) - Q[s,a])
```


# I. SARSA 연습

In [5]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

In [13]:
## Q-table 작성
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n))
print('Q shape :', Q.shape)

rewards_sarsa = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False
    a0 = env.action_space.sample()    # 처음엔 랜덤무브

    while True :
        ## a0를 이용해 움직이고
        ## s1, r1, done, _를 return 받는 코드를 짜자.
        s1, r1, done, _ = env.step(a0)

        # e-greedy 방법을 구현해두자.
        # 행동을 선택하여 a1에 선언하자.
        if np.random.uniform() < epsilon :
          a1 = env.action_space.sample()
        else :
          greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
          a1 = np.random.choice(greedy_actions)

        # Update 코드를 작성해보자.
        # ( s0, a0, r1, s1, a1 ) 로 부터!
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*Q[s1, a1] - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_sarsa.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state
        a0 = a1 # 다음 루프에선 이것이 직전 action

    if (i+1) % 4000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        for rows in Q.reshape([-1,5,5]):
            print(*rows)

Q shape : (500, 6)
===========  에피소드 : 4000  ============
[0. 0. 0. 0. 0.] [ 0.         -5.85226331 -5.57632641 -2.67124517 -0.15290495] [ 1.24811001 -9.23827628  4.51736187  6.20725575  5.08616288] [ 5.83929123 10.00178902 -1.29793507 -0.72020058  1.87825426] [ -2.10240472  -0.77315131   3.04629864  -6.86750888 -11.996131  ]
[-12.30379371 -11.72023626 -12.03735387 -15.94219218 -16.86329933] [0. 0. 0. 0. 0.] [  0.         -11.95093706 -12.96862376 -11.35246919 -12.07491496] [-20.6284553  -18.4105625   -7.89200018  -8.72570095  -8.29330337] [ -8.8761366   -9.23845964 -15.51963414  -0.07859239  -5.20668453]
[ -5.0647284   -5.09985864 -10.33107683  -8.38004024  -6.20637006] [ -9.1087698  -10.8980814  -10.11026138 -13.8644907  -10.09219811] [0. 0. 0. 0. 0.] [ 0.         -4.47619476 -8.55125358 -9.90688983 -9.20677425] [-13.91593424 -13.03194851 -12.01180832 -11.98073229 -10.1025452 ]
[-12.09647829 -15.42842368 -15.37367126  -9.32950074 -10.81391736] [ -9.99619202 -10.31949956 -17.51531594 

### 학습 완료된 것 구경
### 무한 루프 주의
위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.

In [14]:
s0 = env.reset()
done = False
# 처음엔 랜덤액션 선택하여 a0에 선언
a0 = env.action_space.sample()

while True :
    env.render()
    # a0를 이용, 환경과 상호작용하여  s1, r1, done, _를 받아온다.
    s1, r1, done, _ = env.step(a0)

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻?

    # 무한 루프 걸린다면 위의 두줄 주석, 아래를 주석 해제
    # if np.random.uniform() < epsilon :  # 무슨 뜻?
    #     a1 = env.action_space.sample()
    # else : 
    #     greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    #     a1 = np.random.choice(greedy_actions) # 무슨 뜻?


    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: 